# Configure Entra app registration

## Install Graph modules

In [4]:
Set-StrictMode -Version Latest
$ErrorActionPreference = "Stop"
$VerbosePreference = "SilentlyContinue"
$InformationPreference = "Continue"

Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process
Install-Module -Name Microsoft.Graph -Scope CurrentUser
Install-Module -Name Microsoft.Graph.Beta -Scope CurrentUser


## Create API app registration

In [11]:
$apiApplicationDisplayName = "epizzas-api"

Write-Information "Authenticating...."
Connect-MgGraph

$apiApplication = Get-MgApplication -Filter "DisplayName eq '$apiApplicationDisplayName'" -ErrorAction SilentlyContinue
if ($null -eq $apiApplication) {
    Write-Information "Creating application $apiApplicationDisplayName"
    $apiApplication = New-MgApplication -DisplayName $apiApplicationDisplayName -SignInAudience "AzureADMyOrg"
}

if ($apiApplication.Api.Oauth2PermissionScopes.Count -eq 0) {
    Write-Information "Setting application permission scopes..."
    $apiApplication = Update-MgApplication -ApplicationId $apiApplication.Id -BodyParameter @{
        Api            = @{        
            Oauth2PermissionScopes = @(
                @{
                    AdminConsentDescription = "Allow the application to access the API on behalf of the signed-in user."
                    AdminConsentDisplayName = "Access API"
                    Id                      = "b73ce2e1-dd44-4cc5-918d-b93b59961d2c"
                    IsEnabled               = $true
                    Type                    = "User"
                    Value                   = "access_as_user"
                    UserConsentDescription  = "Allow the application to access the API on your behalf."
                    UserConsentDisplayName  = "Access API"
                }
            )
        }
        IdentifierUris = @("api://$($apiApplication.AppId)")
    }
}

$user = Get-MgUser -Filter "userPrincipalName eq '$((Get-MgContext).Account)'"
$ownerIds = Get-MgApplicationOwner -ApplicationId $apiApplication.Id | Select-Object -ExpandProperty Id
if ($ownerIds -notcontains $user.Id) {
    Write-Information "Adding application owner..."
    New-MgApplicationOwnerByRef -ApplicationId $apiApplication.Id -BodyParameter @{
        OdataId = "https://graph.microsoft.com/v1.0/directoryObjects/{$($user.Id)}"
    }
}

Authenticating....
Welcome to Microsoft Graph!

Connected via delegated access using 14d82eec-204b-4c2f-b7e8-296a70dab67e
Readme: https://aka.ms/graph/sdk/powershell
SDK Docs: https://aka.ms/graph/sdk/powershell/docs
API Docs: https://aka.ms/graph/docs

NOTE: You can use the -NoWelcome parameter to suppress this message.

Setting application permission scopes...
ParentContainsErrorRecordException: 
Line |
  34 |  $ownerIds = Get-MgApplicationOwner -ApplicationId $apiApplication.Id  …
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The property 'Id' cannot be found on this object. Verify that the property exists.


Error: Command failed: SubmitCode: $apiApplicationDisplayName = "epizzas-api" ...

## Get api application information

In [8]:
@{
    TenantId = Get-MgContext | Select-Object -ExpandProperty TenantId
    ClientId    = $apiApplication.AppId
    Scopes = $apiApplication.Api.Oauth2PermissionScopes.Value
} | ConvertTo-Json

{
  "TenantId": "1c522c6b-bb3b-4eb2-a3ab-21399e001966",
  "Scopes": "user_impersonation",
  "ClientId": "750bee62-af41-42d5-ab65-a8911c78aac2"
}


Error: Command failed: SubmitCode: @{ ...